<a href="https://colab.research.google.com/github/et-tu-atulu/dog_breed_identification/blob/master/VG19_pup_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#extract the zip file here
import os
from zipfile import ZipFile
zipobj=ZipFile(os.path.abspath('.')+'/drive/My Drive/dataset.zip', 'r')
zipobj.extractall()

In [0]:
#Image augmentation

import os
import keras
from keras.preprocessing.image import ImageDataGenerator


image_gen_train = ImageDataGenerator(rescale = 1./255, horizontal_flip = True, zoom_range = 0.2, rotation_range=90)

train_gen = image_gen_train.flow_from_directory(batch_size=32, directory = os.path.abspath('.')+'/train', shuffle = True, target_size =(150, 150), class_mode = 'sparse')

image_gen_test = ImageDataGenerator(rescale = 1./255)

test_gen = image_gen_test.flow_from_directory(batch_size=32, directory=os.path.abspath('.')+'/test', target_size=(150, 150), class_mode='sparse')



In [0]:
#Creating the model and compiling it
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten


base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

    
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


model.summary()

In [0]:
#training the model
EPOCHS = 50
history = model.fit_generator(train_gen,
                    steps_per_epoch=513,         
                    epochs=EPOCHS,
                    validation_data=test_gen,
                    validation_steps=130)